## Fueling Longevity: Protein's Role in Life Expectancy.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
sns.set_style("white")
from importlib import reload
plt=reload(plt)


# color palette

In [ ]:
colors = sns.color_palette(palette='YlOrBr_r') 
colors_r = sns.color_palette(palette='YlOrBr') 

# cleaning

In [ ]:
protein_one = pd.read_csv('/Users/jonathanfarro/Desktop/animal-protein-consumption.csv')

life_ex = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/life-expectancy-hmd-unwpp.csv')

usa_pricing = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/average_prices_for_food.csv')

daily_cals = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-caloric-supply-derived-from-carbohydrates-protein-and-fat.csv')

fruit_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/fruit-consumption-per-capita.csv')

veg_per_cap = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/vegetable-consumption-per-capita/vegetable-consumption-per-capita.csv' )

protein_per_GDP = pd.read_csv('/Users/jonathanfarro/Desktop/ Capstone Files/daily-per-capita-protein-supply-vs-gdp-per-capita.csv')


fruit = fruit_per_cap.rename(columns={ fruit_per_cap.columns[3]:'fruit_kilo_per_cap_per_year'})
fruit['fruit_grams_per_day']=fruit.iloc[:,3] * 1000/ 365 
fruit = fruit.drop(columns=['fruit_kilo_per_cap_per_year'])

In [ ]:
# rename columns to something easier to work with 

protein = protein_one.rename(columns={
                        protein_one.columns[3]:'fish',
                        protein_one.columns[4]:'poultry',
                        protein_one.columns[5]:'pork',
                        protein_one.columns[6]:'beef',
                        protein_one.columns[7]:'sheep/goat',
                        protein_one.columns[8]:'other',
                        protein_one.columns[9]:'eggs',
                        protein_one.columns[10]:'milk'
                       })

In [ ]:
# bulk clean up 
protein = protein.rename(columns=lambda x: x.lower())
protein = protein.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)

life_ex = life_ex.rename(columns = lambda x: x.lower())
life_ex = life_ex.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
life_ex = life_ex.rename(columns = {life_ex.columns[3] : 'life_expectancy(0)'})

protein_per_GDP = protein_per_GDP.rename(columns = lambda x: x.lower())
protein_per_GDP = protein_per_GDP.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)
gdp_population = protein_per_GDP.drop(columns= {protein_per_GDP.columns[3],protein_per_GDP.columns[6]})
gdp_population = gdp_population.rename(columns = { gdp_population.columns[3]: 'gdp_per_capita'}) 


daily_cals = daily_cals.rename(columns = lambda x: x.lower())
daily_cals = daily_cals.map(lambda x:x.lower() if isinstance(x,str) else x).reset_index(drop = True)


fruit = fruit.rename(columns=lambda x: x.lower())
fruit = fruit.map(lambda x: x.lower() if isinstance(x, str) else x).reset_index(drop=True)


# creating a new column to show total_protein intake for a day.
protein['total_protein'] = protein.iloc[:, 3:11].sum(axis = 1).round(2)



In [ ]:
capstone = pd.merge(protein,
         life_ex,
         on= ['entity','code','year'],
         how= 'inner')

#capstone.head()

capstone_2 = pd.merge( capstone, gdp_population, on = ['entity','code','year'],how= 'inner')

main_frame = pd.merge(capstone_2,daily_cals,  on = ['entity','code','year'],how= 'outer')



final_frame = main_frame.rename(columns= { 'daily calorie supply per person that comes from animal protein':'daily_cal_animal_protein',
                                            'daily calorie supply per person that comes from vegetal protein':'daily_cal_veg_protein',
                                            'daily calorie supply per person from fat':'daily_cal_fat',
                                            'daily calorie supply per person from carbohydrates':'daily_cal_carbs'})
final_frame.head()

final_frame.entity.unique()

In [ ]:
#creating an all_cause_mortality column, and more cleaning



final_frame = final_frame[~final_frame['entity'].str.contains(r'\(.*\)')]

final = final_frame
final = final.rename(columns={'life_expectancy(0)':'life_exp','population (historical)':'population'})
final

In [ ]:
#further cleaning.

non_countries_list = ['asia','europe','high-income countries','low-income countries','lower-middle-income countries','oceania','upper-middle-income countries','world']
income_classes= ['low-income countries','high-income countries','lower-middle-income countries','upper-middle-income countries','world']

final_countries = final.loc[~final['entity'].isin(non_countries_list)]

income_class = final.loc[final['entity'].isin(income_classes)]

income_class['total_daily_protein_g'] = income_class['daily_cal_veg_protein']/4 + income_class['daily_cal_animal_protein']/4
income_class['total_animal_protein_g']= income_class['daily_cal_animal_protein'].round() / 4
income_class['total_veg_protein_g']= income_class['daily_cal_veg_protein'].round() / 4
income_class = income_class.loc[income_class['entity']!= 'world']




income_class.reset_index()


In [ ]:
testdrop= final_countries

df = testdrop.dropna()

df['daily_caloric_intake'] = df.iloc[:,15:19].sum(axis=1)
df['non_protein_cals']= df['daily_cal_fat'] + df['daily_cal_carbs'] 
income_class['daily_caloric_intake'] = income_class.iloc[:,15:19].sum(axis=1)



df = pd.merge(df,
         fruit,
         on= ['entity','code','year'],
         how= 'inner')

df['decade'] = (df['year'] // 10) * 10

df['total_daily_protein_g'] = df['daily_cal_veg_protein']/4 + df['daily_cal_animal_protein']/4
df

In [ ]:
country_to_region = {
    'algeria': 'Africa',
    'angola': 'Africa',
    'benin': 'Africa',
    'botswana': 'Africa',
    'burkina faso': 'Africa',
    'burundi': 'Africa',
    'cabo verde': 'Africa',
    'cameroon': 'Africa',
    'central african republic': 'Africa',
    'chad': 'Africa',
    'comoros': 'Africa',
    'congo, democratic republic of the': 'Africa',
    'congo, republic of the': 'Africa',
    "côte d'ivoire": 'Africa',
    'djibouti': 'Africa',
    'egypt': 'Africa',
    'equatorial guinea': 'Africa',
    'eritrea': 'Africa',
    'eswatini': 'Africa',
    'ethiopia': 'Africa',
    'gabon': 'Africa',
    'gambia': 'Africa',
    'ghana': 'Africa',
    'guinea': 'Africa',
    'guinea-bissau': 'Africa',
    'kenya': 'Africa',
    'lesotho': 'Africa',
    'liberia': 'Africa',
    'libya': 'Africa',
    'madagascar': 'Africa',
    'malawi': 'Africa',
    'mali': 'Africa',
    'mauritania': 'Africa',
    'mauritius': 'Africa',
    'morocco': 'Africa',
    'mozambique': 'Africa',
    'namibia': 'Africa',
    'niger': 'Africa',
    'nigeria': 'Africa',
    'rwanda': 'Africa',
    'sao tome and principe': 'Africa',
    'senegal': 'Africa',
    'seychelles': 'Africa',
    'sierra leone': 'Africa',
    'somalia': 'Africa',
    'south africa': 'Africa',
    'south sudan': 'Africa',
    'sudan': 'Africa',
    'tanzania': 'Africa',
    'togo': 'Africa',
    'tunisia': 'Africa',
    'uganda': 'Africa',
    'zambia': 'Africa',
    'zimbabwe': 'Africa',
    'afghanistan': 'Asia',
    'armenia': 'Asia',
    'azerbaijan': 'Asia',
    'bahrain': 'Asia',
    'bangladesh': 'Asia',
    'bhutan': 'Asia',
    'brunei': 'Asia',
    'cambodia': 'Asia',
    'china': 'Asia',
    'cyprus': 'Asia', 
    'georgia': 'Asia',
    'india': 'Asia',
    'indonesia': 'Asia',
    'iran': 'Asia',
    'iraq': 'Asia',
    'israel': 'Asia',
    'japan': 'Asia',
    'jordan': 'Asia',
    'kazakhstan': 'Asia',
    'kuwait': 'Asia',
    'kyrgyzstan': 'Asia',
    'laos': 'Asia',
    'lebanon': 'Asia',
    'malaysia': 'Asia',
    'maldives': 'Asia',
    'mongolia': 'Asia',
    'myanmar': 'Asia',
    'nepal': 'Asia',
    'north korea': 'Asia',
    'oman': 'Asia',
    'pakistan': 'Asia',
    'palestine': 'Asia',
    'philippines': 'Asia',
    'qatar': 'Asia',
    'russia': 'Asia', # Mostly in Asia
    'saudi arabia': 'Asia',
    'singapore': 'Asia',
    'south korea': 'Asia',
    'sri lanka': 'Asia',
    'syria': 'Asia',
    'tajikistan': 'Asia',
    'thailand': 'Asia',
    'timor-leste': 'Asia',
    'turkey': 'Asia', # Mostly in Asia
    'turkmenistan': 'Asia',
    'united arab emirates': 'Asia',
    'uzbekistan': 'Asia',
    'vietnam': 'Asia',
    'yemen': 'Asia',
    'macao':'Asia',
    'albania': 'Europe',
    'andorra': 'Europe',
    'austria': 'Europe',
    'belarus': 'Europe',
    'belgium': 'Europe',
    'bosnia and herzegovina': 'Europe',
    'bulgaria': 'Europe',
    'croatia': 'Europe',
    'czech republic': 'Europe',
    'denmark': 'Europe',
    'estonia': 'Europe',
    'finland': 'Europe',
    'france': 'Europe',
    'germany': 'Europe',
    'greece': 'Europe',
    'hungary': 'Europe',
    'iceland': 'Europe',
    'ireland': 'Europe',
    'italy': 'Europe',
    'kosovo': 'Europe',
    'latvia': 'Europe',
    'liechtenstein': 'Europe',
    'lithuania': 'Europe',
    'luxembourg': 'Europe',
    'malta': 'Europe',
    'moldova': 'Europe',
    'monaco': 'Europe',
    'montenegro': 'Europe',
    'netherlands': 'Europe',
    'north macedonia': 'Europe',
    'norway': 'Europe',
    'poland': 'Europe',
    'portugal': 'Europe',
    'romania': 'Europe',
    'russia': 'Europe', 
    'san marino': 'Europe',
    'serbia': 'Europe',
    'slovakia': 'Europe',
    'slovenia': 'Europe',
    'spain': 'Europe',
    'sweden': 'Europe',
    'switzerland': 'Europe',
    'ukraine': 'Europe',
    'united kingdom': 'Europe',
    'vatican city': 'Europe', 
    'antigua and barbuda': 'North America',
    'bahamas': 'North America',
    'barbados': 'North America',
    'belize': 'North America',
    'canada': 'North America',
    'costa rica': 'North America',
    'cuba': 'North America',
    'dominica': 'North America',
    'dominican republic': 'North America',
    'el salvador': 'North America',
    'grenada': 'North America',
    'guatemala': 'North America',
    'haiti': 'North America',
    'honduras': 'North America',
    'jamaica': 'North America',
    'mexico': 'North America',
    'nicaragua': 'North America',
    'panama': 'North America',
    'saint kitts and nevis': 'North America',
    'saint lucia': 'North America',
    'saint vincent and the grenadines': 'North America',
    'trinidad and tobago': 'North America',
    'united states': 'North America',
    'australia': 'Oceania',
    'fiji': 'Oceania',
    'kiribati': 'Oceania',
    'marshall islands': 'Oceania',
    'micronesia': 'Oceania',
    'nauru': 'Oceania',
    'new zealand': 'Oceania',
    'palau': 'Oceania',
    'papua new guinea': 'Oceania',
    'samoa': 'Oceania',
    'solomon islands': 'Oceania',
    'tonga': 'Oceania',
    'tuvalu': 'Oceania',
    'vanuatu': 'Oceania',
    'argentina': 'South America',
    'bolivia': 'South America',
    'brazil': 'South America',
    'chile': 'South America',
    'colombia': 'South America',
    'ecuador': 'South America',
    'guyana': 'South America',
    'paraguay': 'South America',
    'peru': 'South America',
    'suriname': 'South America',
    'uruguay': 'South America',
    'venezuela': 'South America',
}

df['region'] = df['entity'].map(country_to_region)

df['entity'] = df['entity'].str.title()

In [ ]:
#df = df.drop(columns =['total_daily_protein','cals_from_protein','total_protein'])

df['total_daily_protein_g'] = df['daily_cal_veg_protein']/4 + df['daily_cal_animal_protein']/4
df['total_veg_protein_g'] = df['daily_cal_veg_protein']/4 
df['total_animal_protein_g'] = df['daily_cal_animal_protein']/4
df['total_daily_fat_g'] = df.daily_cal_fat/9 
df['total_daily_carbs_g']= df.daily_cal_carbs/ 4
df

# correlation chart

In [ ]:
df.columns

In [ ]:
df.iloc[:,9:19].corr()

In [ ]:


# Calculate the correlation matrix
corr = df.iloc[:,9:19].corr()
corr

rs = np.random.RandomState(0)

corr.style.background_gradient(cmap='copper_r')
plt.figure(figsize=(20, 10))  # Adjust figure size as needed
sns.heatmap(corr, annot=True, cmap='copper', fmt=".2f")
plt.title('Correlation Matrix')


In [ ]:
include = df.columns[3:11].to_list() + df.columns[12:14].to_list()


df[include].corr()

In [ ]:
df.fruit_grams_per_day = df.fruit_grams_per_day.round()

# EDA 

In [ ]:
df.year.sort_values().unique()

In [ ]:
df.shape

In [ ]:
df.life_exp.mean()

In [ ]:
df.describe()

In [ ]:
df.life_exp.mean().round()
print(f'The average life expectancy covered in this dataset is {df.life_exp.mean().round()} years old,')


#  What is the correlation between the consumption of food types per capita and life expectancy across different countries, and how has this correlation changed over time? 


# a function to find percentage of countries with above average life expectancy and how much daily protein they eat on average.

In [ ]:
def analyze_above_average_life_exp(df):


  
  average_life_exp = 80

  # Filter 
  above_average_df = df[df['life_exp'] > average_life_exp]

  # Get unique 
  above_average_entities = above_average_df['entity'].unique()

  # Calculate the percentage of entities with above-average life expectancy
  percentage_above_average = (len(above_average_entities) / len(df['entity'].unique())) * 100

  # Calculate the average total daily protein consumption for those entities
  average_protein_consumption = above_average_df['total_daily_protein_g'].mean()

  return percentage_above_average, average_protein_consumption


# Analyze the full DataFrame
percentage, avg_protein = analyze_above_average_life_exp(df)



# Answer to above

In [ ]:
print(f"Percentage of countries with above-average life expectancy: {percentage:.2f}%")
print(f"Average total daily protein consumption for those entities: {avg_protein:.2f} grams")

In [ ]:
plt.figure(figsize=(10, 5))

sns.set_style("white", {"grid.color": ".6", "grid.linestyle": ":"})
sns.scatterplot(data=df, x="total_daily_protein_g", y="life_exp",marker="o",hue='region',s=70,palette=colors)


plt.xlabel("Total Daily Protein Consumption Per Capita (Grams) ",fontweight='bold')
plt.ylabel('Life Expectancy (Years)',fontweight='bold')
plt.tight_layout()
#plt.savefig('yolo.png',transparent=True)


In [ ]:
asia = df.loc[df.region== "Asia"]
europe = df.loc[df.region== "Europe"]
africa = df.loc[df.region== "Africa"]
northamerica = df.loc[df.region== "North America"]
southamerica = df.loc[df.region== "South America"]
oceania= df.loc[df.region== "Oceania"]


asia         = asia.groupby('region').agg({'life_exp':'mean'}).reset_index().round()
europe       = europe.groupby('region').agg({'life_exp':'mean'}).reset_index().round()
northamerica = northamerica.groupby('region').agg({'life_exp':'mean'}).reset_index().round()
southamerica = southamerica.groupby('region').agg({'life_exp':'mean'}).reset_index().round()
oceania      = oceania.groupby('region').agg({'life_exp':'mean'}).reset_index().round()
africa       = africa.groupby('region').agg({'life_exp':'mean'}).reset_index().round()

northsoutham = pd.concat([northamerica,southamerica]).reset_index(drop=True)
asiaeurope = pd.concat([asia,europe]).reset_index(drop=True)
ocean_africa = pd.concat([oceania,africa]).reset_index().round()


oneleft = pd.concat([northsoutham,asiaeurope]).reset_index(drop=True) 
avg_life_exp_all_regions = pd.concat([oneleft,ocean_africa]).reset_index(drop=True)

In [ ]:
plt.subplots(figsize=(10, 6))
ax = sns.barplot(avg_life_exp_all_regions.sort_values(by='life_exp'),x='region',y='life_exp',palette=colors_r,hue='region')


for i in ax.containers:
    ax.bar_label(i,fontsize=10,color='k',)
    plt.xlabel('')
    plt.xticks(fontsize= 12,fontweight='bold')
    plt.ylabel('Life Expectancy at Birth',fontweight='bold')
    plt.title('Average Life Expectancy Ages by Region',fontweight='bold')
    plt.savefig('Life Averages by Region.png',transparent=True);

In [ ]:
pro_df_grouped

In [ ]:
pro_df_grouped = df.groupby(['entity','decade']).agg({'total_veg_protein_g':'sum',
                                           'total_animal_protein_g':'sum',
                                           'life_exp':'mean'}).reset_index()



# Create the 'more_meat' and 'more_veg' df's
average_meat = pro_df_grouped[pro_df_grouped['total_animal_protein_g'] > pro_df_grouped['total_veg_protein_g']].copy()

average_veg = pro_df_grouped[pro_df_grouped['total_veg_protein_g'] > pro_df_grouped['total_animal_protein_g']].copy()

averages_veg = average_veg.groupby('decade').agg({'life_exp':'mean'}).astype(int).reset_index()
averages_meat= average_meat.groupby('decade').agg({'life_exp':'mean'}).astype(int).reset_index()


averages_meat

In [ ]:
fig, ax = plt.subplots()  # Create figure and axes
ax.axis('off')           # Hide the axes
col_widths = [0.3, 0.3]  # Example: 30% for the first column, 70% for the second
custom_labels = ['Decade Range', 'Average Life Expectancy']
table = ax.table(cellText=averages_meat.values, 
                 colLabels=custom_labels, 
                 loc='center', 
                 cellLoc='center', 
                 colWidths=col_widths,
                 colColours=['darkorange', 'darkorange'])

               

table.set_fontsize(11) 
plt.title(" Higher Animal \n Protein Consumption", 
          fontsize=16, fontweight='bold', color='k',y=0.65)  

#plt.savefig('meatlife_table_plot.png', dpi=300, bbox_inches='tight', transparent=True);



In [ ]:
fig, ax = plt.subplots()  # Create figure and axes
ax.axis('off')           # Hide the axes
col_widths = [0.3, 0.3]  # Example: 30% for the first column, 70% for the second
custom_labels = ['Decade Range', 'Average Life Expectancy']
table = ax.table(cellText=averages_veg.values.round().astype('int'), 
                 colLabels=custom_labels, 
                 loc='center', 
                 cellLoc='center', 
                 colWidths=col_widths,
                 colColours=['lightgreen', 'lightgreen'])

               

table.set_fontsize(11) 
plt.title("Higher Vegetebale \n Protein Consumption ", 
          fontsize=16, fontweight='bold', color='k',y=0.65)  
plt.savefig('veglife_table_plot.png', dpi=300, bbox_inches='tight', transparent=True);


# Do countries with a higher ratio vegetables protein consumption compared to animal protein exhibit higher life expectancies, and how does this relationship vary across different regions?

In [ ]:
more_veg

In [ ]:
# Group by entity and take the mean of other columns
df_grouped = df.groupby('entity').agg({'total_daily_protein_g':'mean',
                                       'total_veg_protein_g':'mean',
                                       'total_animal_protein_g':'mean',
                                       'life_exp':'mean',}).reset_index()


# Create the 'more_meat' and 'more_veg' df's
more_meat = df_grouped[df_grouped['total_animal_protein_g'] > df_grouped['total_veg_protein_g']].copy()
more_veg = df_grouped[df_grouped['total_veg_protein_g'] > df_grouped['total_animal_protein_g']].copy()

def calculate_above_69_percentage(df):
    """Calculates the percentage of entities with life expectancy above 69 within the given group."""

    # Get unique entities with life expectancy above 69
    above_69_entities = df[df['life_exp'] > 69].index.unique()  

    # Calculate the percentage 
    percentage_above_69 = (len(above_69_entities) / len(df)) * 100

    return percentage_above_69

# Calculate percentages for both groups
meat_percentage = calculate_above_69_percentage(more_meat)  
veg_percentage = calculate_above_69_percentage(more_veg) 

print(f"Percentage of countries with above average life expectancy in the Higher Animal Protein Consumption group is {meat_percentage:.2f}%")
print(f"Percentage of countries with above average life expectancy in the Higher Vegetable Protein group is {veg_percentage:.2f}%") 


In [ ]:
more_meat = more_meat.loc[more_meat['total_animal_protein_g'] <= more_veg['total_veg_protein_g'].max()] 
more_meat.life_exp = more_meat.life_exp.round() 
more_veg.life_exp  =more_veg.life_exp.round() 
more_meat.life_exp = more_meat.life_exp.astype('int')
more_veg.life_exp = more_veg.life_exp.astype('int')

#  avg life exp
more_meat.life_exp.describe() #64
more_veg.life_exp.describe()  #75

In [ ]:
more_meat['group'] = 'Higher Animal Protein Consumption'
more_veg['group'] = 'Higher Vegetable Protein Consumption'
combined_df = pd.concat([more_meat, more_veg])

 # Create the boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x='group', y='life_exp', data=combined_df, width=0.4,palette=["darkorange","green"])
plt.title('Life Expectancy by Protein Consumption Group',fontweight='bold',fontsize=20)
plt.ylabel('Life Expectancy in Years',fontweight='bold',fontsize=18)
plt.xlabel('')

plt.xticks(fontsize=12);
plt.savefig('replacement_box_animal_veg.png',transparent = True) 

# Does this correlation differ between high-income and low-income countries?

In [ ]:
from importlib import reload
plt=reload(plt)

In [ ]:
# how are these income levels made? avg gdp 


In [ ]:
incomebars = income_class.groupby(['entity']).agg({'life_exp':"mean",
                                                   'total_animal_protein_g':'mean',
                                                   'total_veg_protein_g':'mean',
                                                   'total_daily_protein_g':'mean',
                                                
                                                   })
incomebars = incomebars.reset_index()

incomebars

In [ ]:


# total daily protein intake in calories 

#income_bars.life_exp = income_bars.life_exp.astype('int64')
#income_bars.total_daily_protein = income_bars.total_daily_protein.astype('int64')                                 

incomebars.entity  = incomebars.entity.str.replace('countries','')
incomebars.entity = incomebars.entity.str.title()


incomebars.life_exp = incomebars.life_exp.round().astype('int')
incomebars.entity = incomebars.entity.str.replace('-Income','')

incomebars = incomebars.sort_values(by='life_exp',ascending=True)


In [ ]:
high =incomebars.loc[incomebars['life_exp'] == 75]
uppermiddle = incomebars.loc[incomebars['life_exp'] == 66]
lowermiddle = incomebars.loc[incomebars['life_exp'] == 58]
low= incomebars.loc[incomebars['life_exp'] == 51]

#concat together

income_bars = pd.concat([pd.concat([high,uppermiddle]),pd.concat([lowermiddle,low])])


incomebars

In [ ]:

income_class['entity']=income_class.entity.str.title()
incomebars.entity = incomebars.entity.str.replace('Income','')
incomebars.entity = incomebars.entity.str.replace('Countries','')
incomebars

In [ ]:

plt.figure(figsize=(13, 8))

# Assuming 'colors_r' is defined

ax = sns.barplot(data=incomebars, y='life_exp', x='entity', palette=colors_r, hue='entity') 
plt.xlabel('')
plt.ylabel('Life Expectancy (Years)', fontweight='bold',fontsize=15)
plt.xticks(fontsize=12, fontweight='bold')
plt.title('Average Life Expectancy Based on Income Level', fontweight='bold',fontsize=15)

# Add rounded bold numbers on the bars
for p in ax.patches:
    ax.annotate(f"{int(p.get_height())} Years Old", (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', xytext=(0, 5), textcoords='offset points', 
                fontweight='bold',fontsize=18,color='k')  # Make the text bold

plt.subplots_adjust(wspace=0.5, hspace=0.2) 
plt.savefig('avg_life_income.png',transparent=True)


In [ ]:

incomebars_melted = pd.melt(incomebars, id_vars='entity', 
                           value_vars=['total_animal_protein_g', 'total_veg_protein_g', 'total_daily_protein_g'], 
                           var_name='protein_type', value_name='protein_amount')

ax = sns.barplot(x='entity', y='protein_amount', hue='protein_type', data=incomebars_melted, 
            palette={'total_animal_protein_g': 'darkorange', 'total_veg_protein_g': 'green', 'total_daily_protein_g': 'peru'}, 
            dodge=True)  

plt.xlabel('')
plt.ylabel('Total Daily Protein Grams', fontweight='bold',fontsize=13)
plt.title('Protein Consumption in Grams Per Day By Income Level',fontweight='bold')
plt.xticks()

# Customize the legend
handles, labels = ax.get_legend_handles_labels()  # Get the existing handles and labels
new_labels = ['Animal Protein', 'Vegetable Protein', 'Total Protein']  # Your new labels
plt.legend(handles, new_labels, title='Protein Type') 

plt.tight_layout()
plt.savefig('protein and income.png',transparent=True)

# What animal proteins are most popular amongst countries with above average life expectancy and below average gdp? 

In [ ]:
# showing limited access to goods but still getting healthy amounts of protein.

In [ ]:
low_gdp_high_life =df.loc[(df['gdp_per_capita']< df['gdp_per_capita'].mean()) & (df['life_exp'] > df['life_exp'].mean())]

In [ ]:
low_gdp_high_life_averages = low_gdp_high_life.groupby('decade').agg({'fish':'mean',
                                                     'poultry':'mean',
                                                     'pork':'mean',
                                                     'beef':'mean',
                                                     'eggs':'mean',
                                                     'milk':'mean'}).astype('int').reset_index()

low_gdp_high_life_averages = low_gdp_high_life_averages.rename(columns=lambda x: x.title())

In [ ]:
labels = low_gdp_high_life_averages.columns[1:]  
sizes = low_gdp_high_life_averages.iloc[0, 1:].values 

# Create the pie chart
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140,textprops = {'fontsize': 12},colors=colors)
ax.axis('equal') 

plt.title('Animal Proteins Consumed by \n Countries with Below Average GDP \n  and  \n Above Average Life Expectancy',fontsize=14 ,x=0.51,y=-.4,fontweight='bold')
plt.savefig('meatpie.png',transparent = True)

# Who's life expectancy above average and what do they eat?

In [ ]:
longest_living=df.loc[df['life_exp']> 70]
longest_life = longest_living.groupby('entity').agg({'life_exp':'mean'}).sort_values(by='life_exp')
longest_life = longest_life.reset_index()

long_life_list = longest_life.entity.tolist() 

longest_life_countries = df.loc[df['entity'].isin(long_life_list)]
longlife_grouped = longest_life_countries.groupby('entity').agg({'fish':'mean',
                                                                 'poultry':'mean',
                                                                 'pork':'mean',
                                                                 'beef':'mean',
                                                                 'sheep/goat':'mean',
                                                                 'eggs':'mean',
                                                                 'milk':'mean',
                                                                 }).reset_index()

longlife_grouped = longlife_grouped.rename(columns=lambda x: x.title())




plt.figure(figsize=(10, 6))
sns.boxplot(longlife_grouped.iloc[:,1:8],palette=colors_r)
plt.xticks(fontsize=10,fontweight='bold')
plt.ylabel('Total Daily Consumption in Grams',fontweight='bold')
plt.xlabel('Protein Type',fontweight='bold')

plt.savefig('livinglong_box.png',transparent=True);

In [ ]:
labels = longlife_grouped.columns[1:]  
sizes = longlife_grouped.iloc[0, 1:].values 

# Create the pie chart
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140,textprops = {'fontsize': 12},colors=colors)
ax.axis('equal') 

plt.title('long livers average diets',fontsize=14 ,x=0.51,y=-.4,fontweight='bold')
plt.savefig('longlifersdiet.png',transparent=True)

In [ ]:
kong = df.loc[df['entity']=='Hong Kong']

In [ ]:
kongs = kong.groupby('entity').agg({'fish':'sum',
                            'poultry':'sum',
                            'pork':'sum',
                            'beef':'sum',
                            'sheep/goat':'sum',
                            'eggs':'sum',
                            'milk':'sum',}).reset_index()

kongs[:1]

In [ ]:
labels = kongs.columns[1:]  
sizes = kongs.iloc[0, 1:].values.astype(int)

# Create the pie chart
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140,textprops = {'fontsize': 12},colors=colors)
ax.axis('equal') 

plt.title('long livers average diets',fontsize=14 ,x=0.51,y=-.4,fontweight='bold')
plt.savefig('hong kong max age 85.png',transparent=True)